# Experiment 5: LR Scheduler

- StepLR
- ExponentialLR
- CosineAnnealingLR
- CosineAnnealingWarmRestarts

In [1]:
import os
import sys
import pickle
import glob
import time
from tqdm import tqdm
from collections import Counter

# scikit-learn
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

# Data preprocessing
import cv2
import numpy as np
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
%matplotlib inline

# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import datasets, transforms
torch.manual_seed(0)
print(f'PyTorch version: {torch.__version__}')

# device setting
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'This notebook use {device}')

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

PyTorch version: 1.7.1
This notebook use cuda:0


In [2]:
# 파일 경로 사용자 정의
class path:
    data = '/opt/ml/input/original_data'
    train = f'{data}/train'
    train_img = f'{train}/images'
    train_df = f'{train}/train.csv'
    test = f'{data}/eval'
    test_img = f'{test}/images'
    test_df = f'{test}/info.csv'

In [3]:
BATCH_SIZE = 16
NUM_WORKERS = 2
LEARNING_RATE = 1e-4
EPOCHS = 3

## 1. Dataset

In [4]:
class MaskDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def set_transform(self, transform):
        self.transform = transform
        
    def __getitem__(self, idx):
        data = self.df.iloc[idx]
        target = data.target
        image = Image.open(data.path)
        
        if self.transform:
            image = self.transform(image)
            
        return image, target
    
    def __len__(self):
        return len(self.df)

In [5]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [6]:
train_transforms = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
    AddGaussianNoise(0., 1.),
])

In [7]:
valid_transforms = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

## 2. Modeling

In [8]:
import math
from typing import TYPE_CHECKING, Any, Callable, Optional

import torch
import torch.optim

if TYPE_CHECKING:
    from torch.optim.optimizer import _params_t
else:
    _params_t = Any
    
class MADGRAD(torch.optim.Optimizer):
    """
    MADGRAD_: A Momentumized, Adaptive, Dual Averaged Gradient Method for Stochastic 
    Optimization.
    .. _MADGRAD: https://arxiv.org/abs/2101.11075
    MADGRAD is a general purpose optimizer that can be used in place of SGD or
    Adam may converge faster and generalize better. Currently GPU-only.
    Typically, the same learning rate schedule that is used for SGD or Adam may
    be used. The overall learning rate is not comparable to either method and
    should be determined by a hyper-parameter sweep.
    MADGRAD requires less weight decay than other methods, often as little as
    zero. Momentum values used for SGD or Adam's beta1 should work here also.
    On sparse problems both weight_decay and momentum should be set to 0.
    Arguments:
        params (iterable): 
            Iterable of parameters to optimize or dicts defining parameter groups.
        lr (float): 
            Learning rate (default: 1e-2).
        momentum (float): 
            Momentum value in  the range [0,1) (default: 0.9).
        weight_decay (float): 
            Weight decay, i.e. a L2 penalty (default: 0).
        eps (float): 
            Term added to the denominator outside of the root operation to improve numerical stability. (default: 1e-6).
    """

    def __init__(
        self, params: _params_t, lr: float = 1e-2, momentum: float = 0.9, weight_decay: float = 0, eps: float = 1e-6,
    ):
        if momentum < 0 or momentum >= 1:
            raise ValueError(f"Momentum {momentum} must be in the range [0,1]")
        if lr <= 0:
            raise ValueError(f"Learning rate {lr} must be positive")
        if weight_decay < 0:
            raise ValueError(f"Weight decay {weight_decay} must be non-negative")
        if eps < 0:
            raise ValueError(f"Eps must be non-negative")

        defaults = dict(lr=lr, eps=eps, momentum=momentum, weight_decay=weight_decay)
        super().__init__(params, defaults)

    @property
    def supports_memory_efficient_fp16(self) -> bool:
        return False

    @property
    def supports_flat_params(self) -> bool:
        return True

    def step(self, closure: Optional[Callable[[], float]] = None) -> Optional[float]:
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        # step counter must be stored in state to ensure correct behavior under
        # optimizer sharding
        if 'k' not in self.state:
            self.state['k'] = torch.tensor([0], dtype=torch.long)
        k = self.state['k'].item()

        for group in self.param_groups:
            eps = group["eps"]
            lr = group["lr"] + eps
            decay = group["weight_decay"]
            momentum = group["momentum"]

            ck = 1 - momentum
            lamb = lr * math.pow(k + 1, 0.5)

            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]

                if "grad_sum_sq" not in state:
                    state["grad_sum_sq"] = torch.zeros_like(p.data).detach()
                    state["s"] = torch.zeros_like(p.data).detach()
                    if momentum != 0:
                        state["x0"] = torch.clone(p.data).detach()

                if momentum != 0.0 and grad.is_sparse:
                    raise RuntimeError("momentum != 0 is not compatible with sparse gradients")

                grad_sum_sq = state["grad_sum_sq"]
                s = state["s"]

                # Apply weight decay
                if decay != 0:
                    if grad.is_sparse:
                        raise RuntimeError("weight_decay option is not compatible with sparse gradients")

                    grad.add_(p.data, alpha=decay)

                if grad.is_sparse:
                    grad = grad.coalesce()
                    grad_val = grad._values()

                    p_masked = p.sparse_mask(grad)
                    grad_sum_sq_masked = grad_sum_sq.sparse_mask(grad)
                    s_masked = s.sparse_mask(grad)

                    # Compute x_0 from other known quantities
                    rms_masked_vals = grad_sum_sq_masked._values().pow(1 / 3).add_(eps)
                    x0_masked_vals = p_masked._values().addcdiv(s_masked._values(), rms_masked_vals, value=1)

                    # Dense + sparse op
                    grad_sq = grad * grad
                    grad_sum_sq.add_(grad_sq, alpha=lamb)
                    grad_sum_sq_masked.add_(grad_sq, alpha=lamb)

                    rms_masked_vals = grad_sum_sq_masked._values().pow_(1 / 3).add_(eps)

                    s.add_(grad, alpha=lamb)
                    s_masked._values().add_(grad_val, alpha=lamb)

                    # update masked copy of p
                    p_kp1_masked_vals = x0_masked_vals.addcdiv(s_masked._values(), rms_masked_vals, value=-1)
                    # Copy updated masked p to dense p using an add operation
                    p_masked._values().add_(p_kp1_masked_vals, alpha=-1)
                    p.data.add_(p_masked, alpha=-1)
                else:
                    if momentum == 0:
                        # Compute x_0 from other known quantities
                        rms = grad_sum_sq.pow(1 / 3).add_(eps)
                        x0 = p.data.addcdiv(s, rms, value=1)
                    else:
                        x0 = state["x0"]

                    # Accumulate second moments
                    grad_sum_sq.addcmul_(grad, grad, value=lamb)
                    rms = grad_sum_sq.pow(1 / 3).add_(eps)

                    # Update s
                    s.data.add_(grad, alpha=lamb)

                    # Step
                    if momentum == 0:
                        p.data.copy_(x0.addcdiv(s, rms, value=-1))
                    else:
                        z = x0.addcdiv(s, rms, value=-1)

                        # p is a moving average of z
                        p.data.mul_(1 - ck).add_(z, alpha=ck)


        self.state['k'] += 1
        return loss

### 2.1. StepLR

In [23]:
model1 = torchvision.models.resnet18(pretrained=False)
n_features = model1.fc.in_features
model1.fc = nn.Linear(n_features, 18)
model1 = model1.cuda()

optimizer1 = MADGRAD(model1.parameters(), lr=LEARNING_RATE)
scheduler1 = optim.lr_scheduler.StepLR(optimizer1, step_size=3, gamma=0.1)
criterion1 = nn.CrossEntropyLoss().to(device)

### 2.2. ExponentialLR

In [37]:
model2 = torchvision.models.resnet18(pretrained=False)
n_features = model2.fc.in_features
model2.fc = nn.Linear(n_features, 18)
model2 = model2.cuda()

optimizer2 = MADGRAD(model2.parameters(), lr=LEARNING_RATE)
scheduler2 = optim.lr_scheduler.ExponentialLR(optimizer2, gamma=0.1)
criterion2 = nn.CrossEntropyLoss().to(device)

### 2.3 CosineAnnealingLR

In [38]:
model3 = torchvision.models.resnet18(pretrained=False)
n_features = model3.fc.in_features
model3.fc = nn.Linear(n_features, 18)
model3 = model3.cuda()

optimizer3 = MADGRAD(model3.parameters(), lr=LEARNING_RATE)
scheduler3 = optim.lr_scheduler.CosineAnnealingLR(optimizer3, T_max=3, eta_min=0)
criterion3 = nn.CrossEntropyLoss().to(device)

### 2.4. CosineAnnealingWarmRestarts

In [39]:
model4 = torchvision.models.resnet18(pretrained=False)
n_features = model4.fc.in_features
model4.fc = nn.Linear(n_features, 18)
model4 = model4.cuda()

optimizer4 = MADGRAD(model4.parameters(), lr=LEARNING_RATE)
scheduler4 = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer4, T_0=3, T_mult=1, eta_min=1e-5)
criterion4 = nn.CrossEntropyLoss().to(device)

### 2.5 CyclicLR

In [44]:
model5 = torchvision.models.resnet18(pretrained=False)
n_features = model5.fc.in_features
model5.fc = nn.Linear(n_features, 18)
model5 = model5.cuda()

optimizer5 = MADGRAD(model4.parameters(), lr=LEARNING_RATE)
scheduler5 = optim.lr_scheduler.CyclicLR(optimizer5, 
                                         base_lr=1e-6,
                                         step_size_up=5,
                                         max_lr=1e-4,
                                         mode='exp_range',
                                         gamma=0.995,
                                        )
criterion5 = nn.CrossEntropyLoss().to(device)

## 3. Training

In [28]:
def test_eval(model, valid_dataset):
    model.eval()
    with torch.no_grad():
        y_true, y_pred = [], []
        for image, label in tqdm(valid_dataset):
            X = image.float().to(device)
            y = label.item()
            _, pred = torch.max(model(X), 1)
            pred = pred.item()
            y_true.append(y)
            y_pred.append(pred)
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        f1 = f1_score(y_true, y_pred, average='macro')
        accuracy = accuracy_score(y_true, y_pred)
    model.train()
    return f1, accuracy

In [29]:
def train_model(train, test, model, criterion, optimizer, scheduler, print_every=1):
    print(f"============ Training Starts! ============")
    best_accuracy = 0
    for epoch in range(EPOCHS):
        loss_sum = 0
        for images, label in tqdm(train):
            X = images.float().to(device)
            y = label.to(device)
            
            y_pred = model(X)
            loss = criterion(y_pred, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_sum += loss
            
        # lr 감소
        scheduler.step()
            
        # 평가지표 출력
        if ((epoch % print_every) == 0) or (epoch == (EPOCHS - 1)):
            loss_avg = loss_sum / len(train)
            f1, accuracy = test_eval(model, test)
            print(f">> epoch:[{epoch + 1}/{EPOCHS}] cost: {loss_avg:5.3f} test_accuracy: {accuracy:5.3f} test_f1_score: {f1:5.3f}")
            
    print(f"============ Training Done! ============")

In [19]:
def cross_validation(df, model, criterion, optimizer, k_folds=5):
    skf = StratifiedKFold(n_splits=5)
    for n_iter, (train_idx, valid_idx) in enumerate(skf.split(df, df.target), start=1):
        print(f'>> Cross Validation {n_iter} Starts!')
        train, valid = df.loc[train_idx], df.loc[valid_idx]
        train_dataset, valid_dataset = MaskDataset(train), MaskDataset(valid)
        
        # augmentation 설정
        train_dataset.set_transform(train_transforms)
        valid_dataset.set_transform(valid_transforms)
        
        # DataLoader 생성
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
        valid_loader = DataLoader(valid_dataset, shuffle=False)
        
        train_model(train_loader, valid_loader, model, criterion, optimizer)
        print()

In [33]:
def cross_validation(df, model, criterion, optimizer, scheduler, k_folds=5):
    skf = StratifiedKFold(n_splits=5)
    for n_iter, (train_idx, valid_idx) in enumerate(skf.split(df, df.target), start=1):
        print(f'>> Cross Validation {n_iter} Starts!')
        train, valid = df.loc[train_idx], df.loc[valid_idx]
        train_dataset, valid_dataset = MaskDataset(train), MaskDataset(valid)
        
        # augmentation 설정
        train_dataset.set_transform(train_transforms)
        valid_dataset.set_transform(valid_transforms)
        
        # DataLoader 생성
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
        valid_loader = DataLoader(valid_dataset, shuffle=False)
        
        train_model(train_loader, valid_loader, model, criterion, optimizer, scheduler)
        print()

In [34]:
df = pd.read_csv(f'{path.train}/train_modified.csv')[['path', 'target']]

### 3.1. StepLR

In [40]:
cross_validation(df, model1, criterion1, optimizer1, scheduler1)

  0%|          | 0/945 [00:00<?, ?it/s]

>> Cross Validation 1 Starts!
============ Training Starts! ============


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[1/3] cost: 0.179 test_accuracy: 0.926 test_f1_score: 0.863


  0%|          | 0/945 [00:00<?, ?it/s]

...

>> epoch:[1/3] cost: 0.086 test_accuracy: 0.975 test_f1_score: 0.961


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[2/3] cost: 0.062 test_accuracy: 0.968 test_f1_score: 0.938


100%|██████████| 3780/3780 [01:23<00:00, 45.46it/s]


>> epoch:[3/3] cost: 0.073 test_accuracy: 0.971 test_f1_score: 0.957
============ Training Done! ============



### 3.2. ExponentialLR

In [41]:
cross_validation(df, model2, criterion2, optimizer2, scheduler2)

  0%|          | 0/945 [00:00<?, ?it/s]

>> Cross Validation 1 Starts!
============ Training Starts! ============


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[1/3] cost: 1.558 test_accuracy: 0.643 test_f1_score: 0.439


  0%|          | 0/945 [00:00<?, ?it/s]

...

>> epoch:[1/3] cost: 0.417 test_accuracy: 0.810 test_f1_score: 0.618


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[2/3] cost: 0.383 test_accuracy: 0.792 test_f1_score: 0.594


100%|██████████| 3780/3780 [00:50<00:00, 74.82it/s]


>> epoch:[3/3] cost: 0.359 test_accuracy: 0.791 test_f1_score: 0.608
============ Training Done! ============



### 3.3. CosineAnnealingLR

In [42]:
cross_validation(df, model3, criterion3, optimizer3, scheduler3)

  0%|          | 0/945 [00:00<?, ?it/s]

>> Cross Validation 1 Starts!
============ Training Starts! ============


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[1/3] cost: 1.517 test_accuracy: 0.603 test_f1_score: 0.370


  0%|          | 0/945 [00:00<?, ?it/s]

...

>> epoch:[1/3] cost: 0.412 test_accuracy: 0.829 test_f1_score: 0.755


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[2/3] cost: 0.265 test_accuracy: 0.881 test_f1_score: 0.784


100%|██████████| 3780/3780 [01:35<00:00, 39.77it/s]


>> epoch:[3/3] cost: 0.133 test_accuracy: 0.917 test_f1_score: 0.861
============ Training Done! ============



### 3.4. CosineAnnealingWarmRestarts

In [43]:
cross_validation(df, model4, criterion4, optimizer4, scheduler4)

  0%|          | 0/945 [00:00<?, ?it/s]

>> Cross Validation 1 Starts!
============ Training Starts! ============


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[1/3] cost: 1.524 test_accuracy: 0.571 test_f1_score: 0.392


  0%|          | 0/945 [00:00<?, ?it/s]

...

>> epoch:[1/3] cost: 0.340 test_accuracy: 0.883 test_f1_score: 0.800


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[2/3] cost: 0.203 test_accuracy: 0.902 test_f1_score: 0.847


100%|██████████| 3780/3780 [01:32<00:00, 40.70it/s]


>> epoch:[3/3] cost: 0.105 test_accuracy: 0.930 test_f1_score: 0.867
============ Training Done! ============



### 3.5 CyclicLR

In [45]:
cross_validation(df, model5, criterion5, optimizer5, scheduler5)

  0%|          | 0/945 [00:00<?, ?it/s]

>> Cross Validation 1 Starts!
============ Training Starts! ============


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[1/3] cost: 3.172 test_accuracy: 0.033 test_f1_score: 0.015


  0%|          | 0/945 [00:00<?, ?it/s]

...

>> epoch:[3/3] cost: 3.173 test_accuracy: 0.031 test_f1_score: 0.014
============ Training Done! ============

>> Cross Validation 3 Starts!
============ Training Starts! ============


  0%|          | 0/945 [00:00<?, ?it/s]

>> epoch:[1/3] cost: 3.172 test_accuracy: 0.029 test_f1_score: 0.012


 32%|███▏      | 302/945 [00:34<01:13,  8.74it/s]


KeyboardInterrupt: 